<a href="https://colab.research.google.com/github/ngolla/video-captioning/blob/master/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from pathlib import Path
import numpy as np
import pickle

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
drive_path=Path('/content/drive/My Drive')
list(drive_path.glob('*'))

[PosixPath('/content/drive/My Drive/Colab Notebooks'),
 PosixPath('/content/drive/My Drive/YouTubeClips.tar'),
 PosixPath('/content/drive/My Drive/video_corpus.csv'),
 PosixPath('/content/drive/My Drive/.ipynb_checkpoints'),
 PosixPath('/content/drive/My Drive/VideoArrays'),
 PosixPath('/content/drive/My Drive/splitdata'),
 PosixPath('/content/drive/My Drive/splitset')]

In [5]:
splitset_path=drive_path.joinpath('splitset')
list(splitset_path.glob('*'))

[PosixPath('/content/drive/My Drive/splitset/train.dat'),
 PosixPath('/content/drive/My Drive/splitset/test.dat'),
 PosixPath('/content/drive/My Drive/splitset/tokens_word_index.txt'),
 PosixPath('/content/drive/My Drive/splitset/tokens_index_word.txt'),
 PosixPath('/content/drive/My Drive/splitset/model_25.h5'),
 PosixPath('/content/drive/My Drive/splitset/encoder.h5'),
 PosixPath('/content/drive/My Drive/splitset/decoder.h5')]

In [11]:
tokens=pickle.load(open(file=splitset_path.joinpath('tokens_word_index.txt'), mode='rb'))
tokens.items(), len(tokens.items())

(dict_items([('<unk>', 1), ('<start>', 2), ('<end>', 3), ('is', 4), ('the', 5), ('man', 6), ('on', 7), ('woman', 8), ('playing', 9), ('in', 10), ('of', 11), ('are', 12), ('with', 13), ('and', 14), ('to', 15), ('person', 16), ('into', 17), ('an', 18), ('his', 19), ('dog', 20), ('cat', 21), ('girl', 22), ('some', 23), ('boy', 24), ('cutting', 25), ('someone', 26), ('two', 27), ('walking', 28), ('riding', 29), ('men', 30), ('from', 31), ('doing', 32), ('people', 33), ('lady', 34), ('up', 35), ('at', 36), ('slicing', 37), ('her', 38), ('down', 39), ('eating', 40), ('water', 41), ('running', 42), ('dancing', 43), ('cooking', 44), ('making', 45), ('women', 46), ('ball', 47), ('group', 48), ('off', 49), ('jumping', 50), ('car', 51), ('something', 52), ('small', 53), ('one', 54), ('out', 55), ('how', 56), ('horse', 57), ('putting', 58), ('while', 59), ('performing', 60), ('for', 61), ('boys', 62), ('its', 63), ('baby', 64), ('shooting', 65), ('singing', 66), ('guitar', 67), ('peeling', 68), ('

In [17]:
vocab_size=len(tokens)+1
reverse_word_map = dict(map(reversed, tokens.items()))#instead we can also load "tokens_index_word.txt" file

In [35]:
# Function takes a tokenized sentence and returns the words
def sequence_to_text(all_indices):
    # Looking up words in dictionary
    all_words = [reverse_word_map.get(eachWord) for eachWord in all_indices if eachWord]
    return(all_words)
def predict_sequence(inf_enc, inf_dec, source, n_steps, cardinality):
    # encode
    state = inf_enc.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = inf_dec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0, 0, :])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    
    out = np.array(output).argmax(axis = 1)
    
    return ' '.join(sequence_to_text(out))


In [19]:
from keras import models
model=models.load_model(splitset_path.joinpath('model_25.h5'))
enc=models.load_model(splitset_path.joinpath('encoder.h5'))
dec=models.load_model(splitset_path.joinpath('decoder.h5'))

In [60]:
vd_paths=list(drive_path.joinpath('VideoArrays').glob('*'))[1960:1970]

In [32]:
from tqdm import tqdm

In [63]:
def load_videos_as_numpy(video_paths):
  video_np={}
  for eachVideo in tqdm(video_paths):
    video_np[eachVideo.stem]=np.load(eachVideo)
  return video_np


In [88]:
videos=load_videos_as_numpy(vd_paths)

100%|██████████| 10/10 [00:00<00:00, 157.16it/s]


In [101]:
import pandas as pd

all_anns=pd.read_csv(drive_path.joinpath('processed.csv'))

In [106]:
predictions=[]
keys=[]
for i, (key,val) in tqdm(enumerate(videos.items())):
  predictions.append(predict_sequence(enc, dec, np.array([val]), 35, vocab_size).split())
  keys.append(key)

10it [00:16,  1.61s/it]


In [ ]:
import nltk
nltk.translate.bleu_score.corpus_bleu(output, predictions)